In [1]:
import os
import json

In [2]:
# 파일 위치 지정
keys_file_path = os.path.join('data', 'api_keys.txt')

# 파일에서 API 키를 로드하는 함수
def load_api_keys(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        keys = json.load(file)
    return keys

# API 키 사용
api_keys = load_api_keys(keys_file_path)
pinecone_keys = api_keys['pinecone_key1']

In [6]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 연결 및 index 설정
pc = Pinecone(api_key=pinecone_keys)
index = pc.Index('minilm')

index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.41754,
 'namespaces': {'': {'vector_count': 41754}},
 'total_vector_count': 41754}

In [7]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# LangChain이 지원하는 다른 채팅 모델을 사용합니다. 여기서는 Ollama를 사용합니다.
llm = ChatOllama(model="llama2")

In [8]:
from langchain.prompts import PromptTemplate

template = """Information: {retrieval}
Question: {query}"""

prompt = PromptTemplate.from_template(template=template)

In [9]:
# LangChain 표현식 언어 체인 구문을 사용합니다.
# LCEL에 대한 자세한 내용은 https://python.langchain.com/docs/expression_language/why 에서 확인할 수 있습니다.
chain = prompt | llm | StrOutputParser()

In [10]:
query = "Which country likes tea?"

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

response = model.encode(query)

In [12]:
matches = index.query(
    vector=response.data[0].embedding,
    top_k=3,
    include_values=False,
    include_metadata=True,
)

AttributeError: 'float' object has no attribute 'embedding'

In [ ]:
# 찾아온 벡터들의 토큰화된 metadata들 다시 이어 붙임
assertions_list = [match['metadata']['assertion'].strip() for match in matches['matches']]

print(assertions_list)

['Tea is a popular drink in Central Asia, and is enjoyed by people of all ages.', 'Central Asia is the land where fermented milk really came into vogue.', 'Kefir is a traditional yogurt culture from Central Asia that has spread to other parts of the world.']


In [ ]:
result = llm_chain.invoke({"retrieval": assertions_list, "query":query})

Central Asia